#### update

- 3/31
    - wav_annot_split: segment 인덱스 넘어가는 부분 수정
    - filter 추가
- 3/31 데이터셋 다시 생성
    1. ~normal 있는거 + size 224~
    2. ~normal 없는거 + size 224~
    3. ~normal 있는거 + size 1280, 720~
    4. ~normal 없는거 + size 1280, 720~
<br>
<br>
       
- 위에서 성능 조금이라도 보이면, 그걸로
    6. random shuffle
    7. with augmentation


### Import

In [2]:
import os
current_directory = os.getcwd()
print(current_directory)

/home/sogaksa123/AIFFEL_THON/data/dataset5_0403


In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sys
print(sys.version)
print(pd.__version__)
print(np.__version__)

import xml.etree.cElementTree as ET
import glob
import os
import json
import random
import shutil

from PIL import Image, ImageOps

3.10.13 (main, Sep 11 2023, 13:44:35) [GCC 11.2.0]
2.2.1
1.26.4


### Review Data

In [28]:
# This is a label from the annotation folder. This label is .xml format.

# 1. 224
# xml_files_dir = '/home/sogaksa123/AIFFEL_THON/data/dataset5_0403/with_normal_224/xml_files/'
# png_files_dir = '/home/sogaksa123/AIFFEL_THON/data/dataset5_0403/with_normal_224/png_files/'

# 2. 1280
# xml_files_dir = '/home/sogaksa123/AIFFEL_THON/data/dataset5_0403/with_normal_1280/xml_files/'
# png_files_dir = '/home/sogaksa123/AIFFEL_THON/data/dataset5_0403/with_normal_1280/png_files/'

# 3. filter 224
# xml_files_dir = '/home/sogaksa123/AIFFEL_THON/data/dataset5_0403/filtering5_224/xml_files/'
# png_files_dir = '/home/sogaksa123/AIFFEL_THON/data/dataset5_0403/filtering5_224/png_files/'

# 4. filter 1280
xml_files_dir = '/home/sogaksa123/AIFFEL_THON/data/dataset5_0403/filtering5_1280/xml_files/'
png_files_dir = '/home/sogaksa123/AIFFEL_THON/data/dataset5_0403/filtering5_1280/png_files/'


sample_xml = xml_files_dir+'107_2b3_Ar_mc_AKGC417L_1.xml'
with open(sample_xml) as f:
    contents = f.read()
    print(contents)

<?xml version="1.0" ?>
<annotation>
    <folder>png_files</folder>
    <filename>107_2b3_Ar_mc_AKGC417L_1.png</filename>
    <path>/home/sogaksa123/AIFFEL_THON/data/dataset5_0403/filtering5_1280/png_files/107_2b3_Ar_mc_AKGC417L_1.png</path>
    <source>
        <database>ICBHI</database>
    </source>
    <size>
        <width>1280</width>
        <height>720</height>
        <depth>3</depth>
    </size>
    <segmented>0</segmented>
    <object>
        <name>crckles</name>
        <pose>Unspecified</pose>
        <truncated>0</truncated>
        <difficult>0</difficult>
        <bndbox>
            <xmin>0</xmin>
            <ymin>180</ymin>
            <xmax>169</xmax>
            <ymax>702</ymax>
        </bndbox>
    </object>
    <object>
        <name>crckles</name>
        <pose>Unspecified</pose>
        <truncated>0</truncated>
        <difficult>0</difficult>
        <bndbox>
            <xmin>169</xmin>
            <ymin>180</ymin>
            <xmax>645</xmax>
            <y

# Data Loading

In [5]:
## The functions are about translating label data.
def xml_to_yolo_bbox(bbox, w, h):
    # print('bbox, w, h: ', bbox, w, h)
    # print('bbox[i]: ', bbox[0], bbox[1], bbox[2], bbox[3])
    x_center = ((bbox[2] + bbox[0]) / 2) / w
    y_center = ((bbox[3] + bbox[1]) / 2 ) / h # y

    width = (bbox[2] - bbox[0]) / w
    height = (bbox[3] - bbox[1]) / h # y
    # print('[x_center, y_center, width, height]: ', x_center, y_center, width, height)
    return [x_center, y_center, width, height]

def yolo_to_xml_bbox(bbox, w, h):
    # x_center, y_center, width, height
    w_half_len = (bbox[2] + w) / 2
    h_half_len = (bbox[3] + h) / 2

    xmin = int((bbox[0] + w) - w_half_len)
    ymin = int((bbox[1] + h) - h_half_len)
    xmax = int((bbox[0] + w) + w_half_len)
    ymax = int((bbox[1] + h) + h_half_len)

    return [xmin, ymin, xmax, ymax]

In [29]:
# # 1. size = 224, 224
# input_dir = '/home/sogaksa123/AIFFEL_THON/data/dataset5_0403/with_normal_224/xml_files/'
# output_dir = '/home/sogaksa123/AIFFEL_THON/data/dataset5_0403/model/Yolov8/dataset/Yolo8_224/labels/'
# image_dir = '/home/sogaksa123/AIFFEL_THON/data/dataset5_0403/with_normal_224/png_files/'

# # 2. size = 1280, 720
# input_dir = '/home/sogaksa123/AIFFEL_THON/data/dataset5_0403/with_normal_1280/xml_files/'
# output_dir = '/home/sogaksa123/AIFFEL_THON/data/dataset5_0403/model/Yolov8/dataset/Yolo8_1280/labels/'
# image_dir = '/home/sogaksa123/AIFFEL_THON/data/dataset5_0403/with_normal_1280/png_files/'

# # 3. filtering size = 224, 224
# input_dir = '/home/sogaksa123/AIFFEL_THON/data/dataset5_0403/filtering5_224/xml_files/'
# output_dir = '/home/sogaksa123/AIFFEL_THON/data/dataset5_0403/model/Yolov8/dataset/filtered_224/labels/'
# image_dir = '/home/sogaksa123/AIFFEL_THON/data/dataset5_0403/filtering5_224/png_files/'

# # 4. filtering size = 1280, 720
input_dir = '/home/sogaksa123/AIFFEL_THON/data/dataset5_0403/filtering5_1280/xml_files/'
output_dir = '/home/sogaksa123/AIFFEL_THON/data/dataset5_0403/model/Yolov8/dataset/filtered_1280/labels/'
image_dir = '/home/sogaksa123/AIFFEL_THON/data/dataset5_0403/filtering5_1280/png_files/'

## Create .xml to .txt label data. 

In [30]:
# Create .txt label data.
classes = []

if not os.path.exists(output_dir):
    os.makedirs(output_dir)

files = glob.glob(os.path.join(input_dir, "*.xml"))
for fil in files:
    basename = os.path.basename(fil)
    filename = os.path.splitext(basename)[0]
    if not os.path.exists(os.path.join(image_dir, f"{filename}.png")):
        print(f"{filename} image does not exist")
        continue

    result = []

    tree = ET.parse(fil)
    root = tree.getroot()
    width = int(root.find("size").find("width").text)
    height = int(root.find("size").find("height").text)

    for obj in root.findall("object"):
        label = obj.find("name").text

        if label not in classes:
            classes.append(label)

        index = classes.index(label)
        pil_bbox = [int(x.text) for x in obj.find("bndbox")]
        # print('pil_bbox: ', pil_bbox)
        
        yolo_bbox = xml_to_yolo_bbox(pil_bbox, width, height)
        # print('yolo_bbox: ', yolo_bbox)
        # print('\n','\n')
        # break
        bbox_string = " ".join([str(x) for x in yolo_bbox])
        result.append(f"{index} {bbox_string}")
    # break
    if result:

        with open(os.path.join(output_dir, f"{filename}.txt"), "w", encoding = "utf-8") as f:
            f.write("\n".join(result))

with open(f"{output_dir}/classes.txt", "w", encoding = "utf-8") as f:
    f.write(json.dumps(classes))

In [31]:
# Labels
with open(f"{output_dir}/classes.txt") as f:
    contents = f.read()
    print(contents)

["crckles", "wheezes", "normal"]


In [32]:
classes

['crckles', 'wheezes', 'normal']

### Preparing Training Data

In [33]:
# size (224 or 1280)

# v = 224
v = 1280


In [34]:
root = f'/home/sogaksa123/AIFFEL_THON/data/dataset5_0403/model/Yolov8/dataset/Yolo8_{v}/Training/data'

def makedirectory(path):
    if not os.path.exists(path):
        os.makedirs(path)

makedirectory(f'/home/sogaksa123/AIFFEL_THON/data/dataset5_0403/model/Yolov8/dataset/filtered_{v}')
makedirectory(f'/home/sogaksa123/AIFFEL_THON/data/dataset5_0403/model/Yolov8/dataset/filtered_{v}/Training')
makedirectory(f'/home/sogaksa123/AIFFEL_THON/data/dataset5_0403/model/Yolov8/dataset/filtered_{v}/Training/data')


In [35]:
# Create a folder of train dataset

if not os.path.isfile(f"/home/sogaksa123/AIFFEL_THON/data/dataset5_0403/model/Yolov8/dataset/filtered_{v}/Training/data"):
    makedirectory(f'/home/sogaksa123/AIFFEL_THON/data/dataset5_0403/model/Yolov8/dataset/filtered_{v}/Training/data/train')
    makedirectory(f'/home/sogaksa123/AIFFEL_THON/data/dataset5_0403/model/Yolov8/dataset/filtered_{v}/Training/data/val')
    makedirectory(f'/home/sogaksa123/AIFFEL_THON/data/dataset5_0403/model/Yolov8/dataset/filtered_{v}/Training/data/test')
    makedirectory(f'/home/sogaksa123/AIFFEL_THON/data/dataset5_0403/model/Yolov8/dataset/filtered_{v}/Training/data/train/images')
    makedirectory(f'/home/sogaksa123/AIFFEL_THON/data/dataset5_0403/model/Yolov8/dataset/filtered_{v}/Training/data/train/labels')
    makedirectory(f'/home/sogaksa123/AIFFEL_THON/data/dataset5_0403/model/Yolov8/dataset/filtered_{v}/Training/data/test/images')
    makedirectory(f'/home/sogaksa123/AIFFEL_THON/data/dataset5_0403/model/Yolov8/dataset/filtered_{v}/Training/data/test/labels')
    makedirectory(f'/home/sogaksa123/AIFFEL_THON/data/dataset5_0403/model/Yolov8/dataset/filtered_{v}/Training/data/val/images')
    makedirectory(f'/home/sogaksa123/AIFFEL_THON/data/dataset5_0403/model/Yolov8/dataset/filtered_{v}/Training/data/val/labels')

In [36]:
# Material is the names of images in the images folder.

####################### metarial부분 랜덤하게 섞어서 다시 돌려보기 #################################################
metarial = []
for i in sorted(os.listdir("/home/sogaksa123/AIFFEL_THON/data/dataset5_0403/filtering5_224/png_files")):
    string = i[:-4]
    metarial.append(string)

In [37]:
metarial[0:10]

['101_1b1_Al_sc_Meditron_0',
 '101_1b1_Al_sc_Meditron_1',
 '101_1b1_Al_sc_Meditron_2',
 '101_1b1_Pr_sc_Meditron_0',
 '101_1b1_Pr_sc_Meditron_1',
 '101_1b1_Pr_sc_Meditron_2',
 '102_1b1_Ar_sc_Meditron_0',
 '102_1b1_Ar_sc_Meditron_1',
 '102_1b1_Ar_sc_Meditron_2',
 '103_2b2_Ar_mc_LittC2SE_0']

### random.shuffle

In [38]:
# 랜덤 시드 설정
random.seed(42)
# 리스트 내용을 랜덤하게 섞기
material_random = list(metarial)
print(type(metarial))
print(type(material_random))

random.shuffle(material_random)
print(type(material_random))

<class 'list'>
<class 'list'>
<class 'list'>


In [39]:
material_random[:10]

['198_1b5_Pr_mc_AKGC417L_1',
 '203_2p3_Pr_mc_AKGC417L_1',
 '163_2b2_Tc_mc_AKGC417L_1',
 '130_1p4_Pr_mc_AKGC417L_2',
 '213_1p2_Al_mc_AKGC417L_0',
 '186_2b2_Pr_mc_AKGC417L_1',
 '130_3p4_Pl_mc_AKGC417L_2',
 '162_2b3_Pl_mc_AKGC417L_1',
 '221_2b1_Ar_mc_LittC2SE_0',
 '138_1p3_Ll_mc_AKGC417L_1']

In [40]:
metarial[:10]


['101_1b1_Al_sc_Meditron_0',
 '101_1b1_Al_sc_Meditron_1',
 '101_1b1_Al_sc_Meditron_2',
 '101_1b1_Pr_sc_Meditron_0',
 '101_1b1_Pr_sc_Meditron_1',
 '101_1b1_Pr_sc_Meditron_2',
 '102_1b1_Ar_sc_Meditron_0',
 '102_1b1_Ar_sc_Meditron_1',
 '102_1b1_Ar_sc_Meditron_2',
 '103_2b2_Ar_mc_LittC2SE_0']

In [41]:
print(id(metarial))
print(id(material_random))


140282336840640
140282337767424


In [42]:
# print(sorted(metarial[0:10]))
print("Sum of image: ", len(material_random))
train_size = int(len(material_random) * 0.7)
test_size = int(len(material_random) * 0.15)
val_size = int(len(material_random) * 0.15)
print("Sum of the train size: ", train_size)
print("Sum of the test size: ", test_size)
print("Sum of the val size: ", val_size)

Sum of image:  2978
Sum of the train size:  2084
Sum of the test size:  446
Sum of the val size:  446


In [40]:
# # 각 랜덤리스트에 c, w, n 몇개씩 들어가있는지 출력
# mstring_train = material_random[0:train_size]
# mstring_test = material_random[train_size:train_size + test_size]
# mstring_val = material_random[train_size + test_size:]

# print(len(mstring_train))
# print(len(mstring_test))
# print(len(mstring_val))
# '''
# mstring_train[0]+'.png'
# mstring_train[0]+'.txt'
# get_annotation df

# '''
# print(mstring_train[0])
# print(mstring_test[0])
# print(mstring_val[0])


### 각 split에 있는 label 갯수 세기

In [22]:
# label_csv_path = '/home/sogaksa123/AIFFEL_THON/data/label_base_original.csv'

# label_csv = pd.read_csv(label_csv_path)
# label_csv
# # for i, rows


# Data Split

In [43]:
import os
import shutil

def preparingdata(main_txt_file, main_img_file, train_size, test_size, val_size):
    for i in range(0, train_size):
        source_txt = main_txt_file + "/" + material_random[i] + ".txt"
        source_img = main_img_file + "/" + material_random[i] + ".png"

        mstring = material_random[i]
        train_destination_txt = f"/home/sogaksa123/AIFFEL_THON/data/dataset5_0403/model/Yolov8/dataset/filtered_{v}/Training/data/train/labels" + "/" + material_random[i] + ".txt"
        train_destination_png = f"/home/sogaksa123/AIFFEL_THON/data/dataset5_0403/model/Yolov8/dataset/filtered_{v}/Training/data/train/images" + "/" + material_random[i] + ".png"

        shutil.copy(source_txt, train_destination_txt)
        shutil.copy(source_img, train_destination_png)

    for l in range(train_size , train_size + test_size):
        source_txt = main_txt_file + "/" + material_random[l] + ".txt"
        source_img = main_img_file + "/" + material_random[l] + ".png"

        mstring = material_random[l]
        test_destination_txt = f"/home/sogaksa123/AIFFEL_THON/data/dataset5_0403/model/Yolov8/dataset/filtered_{v}/Training/data/test/labels" + "/" + material_random[l] + ".txt"
        test_destination_png = f"/home/sogaksa123/AIFFEL_THON/data/dataset5_0403/model/Yolov8/dataset/filtered_{v}/Training/data/test/images" + "/" + material_random[l] + ".png"

        shutil.copy(source_txt, test_destination_txt)
        shutil.copy(source_img, test_destination_png)

    for n in range(train_size + test_size , train_size + test_size + val_size):
        source_txt = main_txt_file + "/" + material_random[n] + ".txt"
        source_img = main_img_file + "/" + material_random[n] + ".png"

        mstring = material_random[n]
        val_destination_txt = f"/home/sogaksa123/AIFFEL_THON/data/dataset5_0403/model/Yolov8/dataset/filtered_{v}/Training/data/val/labels" + "/" + material_random[n] + ".txt"
        val_destination_png = f"/home/sogaksa123/AIFFEL_THON/data/dataset5_0403/model/Yolov8/dataset/filtered_{v}/Training/data/val/images" + "/" + material_random[n] + ".png"
        
        shutil.copy(source_txt, val_destination_txt)
        shutil.copy(source_img, val_destination_png)
        

In [44]:
#preparinbdata("/kaggle/working/labels", "/kaggle/input/face-mask-detection/images", 603, 150, 100)

# # 1. size = 224, 224
# preparingdata(main_txt_file = "/home/sogaksa123/AIFFEL_THON/data/dataset5_0403/model/Yolov8/dataset/Yolo8_224/labels/",
#               main_img_file = "/home/sogaksa123/AIFFEL_THON/data/dataset5_0403/with_normal_224/png_files",
#               train_size = train_size,
#               test_size = test_size,
#               val_size = val_size)

# # 2. size = 1280, 720
# preparingdata(main_txt_file = "/home/sogaksa123/AIFFEL_THON/data/dataset5_0403/model/Yolov8/dataset/Yolo8_1280/labels/",
#               main_img_file = "/home/sogaksa123/AIFFEL_THON/data/dataset5_0403/with_normal_1280/png_files",
#               train_size = train_size,
#               test_size = test_size,
#               val_size = val_size)

# 3. filtering size = 224, 224
# preparingdata(main_txt_file = "/home/sogaksa123/AIFFEL_THON/data/dataset5_0403/model/Yolov8/dataset/filtered_224/labels/",
#               main_img_file = "/home/sogaksa123/AIFFEL_THON/data/dataset5_0403/filtering5_224/png_files",
#               train_size = train_size,
#               test_size = test_size,
#               val_size = val_size)

# # 4. filtering size = 1280, 720
preparingdata(main_txt_file = "/home/sogaksa123/AIFFEL_THON/data/dataset5_0403/model/Yolov8/dataset/filtered_1280/labels/",
              main_img_file = "/home/sogaksa123/AIFFEL_THON/data/dataset5_0403/filtering5_1280/png_files",
              train_size = train_size,
              test_size = test_size,
              val_size = val_size)


In [45]:
train_data = len(os.listdir(f"/home/sogaksa123/AIFFEL_THON/data/dataset5_0403/model/Yolov8/dataset/filtered_{v}/Training/data/train/images"))
val_data = len(os.listdir(f"/home/sogaksa123/AIFFEL_THON/data/dataset5_0403/model/Yolov8/dataset/filtered_{v}/Training/data/val/images"))
test_data = len(os.listdir(f"/home/sogaksa123/AIFFEL_THON/data/dataset5_0403/model/Yolov8/dataset/filtered_{v}/Training/data/test/images"))

print(f"train_data: {train_data}")
print(f"val_data: {val_data}")
print(f"test_data: {test_data}")

train_data: 2084
val_data: 446
test_data: 446


In [46]:
# Create the data.yaml. I am going to use it on the train with yolov9.

nc = 3 # with normal
names = ["crackles", "wheezes", "normal"]

# nc = 2 # without normal
# names = ["crackles", "wheezes"]

yaml_text = f"""train: /home/sogaksa123/AIFFEL_THON/data/dataset5_0403/model/Yolov8/dataset/filtered_{v}/Training/data/train/images/
val: /home/sogaksa123/AIFFEL_THON/data/dataset5_0403/model/Yolov8/dataset/filtered_{v}/Training/data/val/images/

nc: {nc}
names: {names}"""

with open(f"/home/sogaksa123/AIFFEL_THON/data/dataset5_0403/model/Yolov8/dataset/filtered_{v}/data_filtered{v}.yaml", 'w') as file:
    file.write(yaml_text)

In [47]:
with open(f"/home/sogaksa123/AIFFEL_THON/data/dataset5_0403/model/Yolov8/dataset/filtered_{v}/data_filtered{v}.yaml") as f:
    contents = f.read()
    print(contents)

train: /home/sogaksa123/AIFFEL_THON/data/dataset5_0403/model/Yolov8/dataset/filtered_1280/Training/data/train/images/
val: /home/sogaksa123/AIFFEL_THON/data/dataset5_0403/model/Yolov8/dataset/filtered_1280/Training/data/val/images/

nc: 3
names: ['crackles', 'wheezes', 'normal']


In [48]:
len(os.listdir(f"/home/sogaksa123/AIFFEL_THON/data/dataset5_0403/model/Yolov8/dataset/filtered_{v}/labels"))

2979